In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
#Task 2
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Feature engineering for lookalike model
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': 'nunique'
}).reset_index()

# Normalize features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(customer_features[['TotalValue', 'Quantity', 'ProductID']])

# Compute similarity scores
similarity_matrix = cosine_similarity(normalized_features)

# Get top 3 similar customers for first 20 customers
lookalike_results = {}
for idx, customer_id in enumerate(customer_features['CustomerID'][:20]):
    similar_indices = similarity_matrix[idx].argsort()[-4:-1][::-1]
    lookalikes = [(customer_features['CustomerID'][i], similarity_matrix[idx][i]) for i in similar_indices]
    lookalike_results[customer_id] = lookalikes

# Create Lookalike.csv
lookalike_df = pd.DataFrame({
    'CustomerID': list(lookalike_results.keys()),
    'Lookalikes': [str(lookalike_results[customer]) for customer in lookalike_results]
})
lookalike_df.to_csv('Lookalike.csv', index=False)
